In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
data = pd.read_csv("/content/drive/My Drive/vital_sign_fmt.csv")
# sha_subset = pd.read_csv("/content/drive/My Drive/sha_subset.csv")
# paper = pd.read_csv("/content/drive/My Drive/metadata_2021-04-26.csv")


In [ ]:
data

,Unnamed: 0,subject_id,hadm_id,deathtime,stay_id,charttime,temperature,heartrate,resprate,o2sat,sbp,dbp,rhythm,pain,hospital_expire_flag
0,0,10000032,22595853,NaN,33258284,2180-05-06 23:04:00,97.7,79.0,16.0,98.0,107.0,60.0,NaN,0,0
1,1,10000032,22841357,NaN,38112554,2180-06-26 18:42:00,97.9,76.0,18.0,95.0,95.0,64.0,NaN,5,0
2,2,10000032,22841357,NaN,38112554,2180-06-26 20:54:00,97.9,86.0,17.0,93.0,96.0,57.0,NaN,NaN,0
3,3,10000032,25742920,NaN,35968195,2180-08-05 23:50:00,98.5,96.0,17.0,100.0,102.0,58.0,NaN,NaN,0
4,4,10000032,25742920,NaN,35968195,2180-08-06 01:07:00,98.1,91.0,18.0,99.0,98.0,60.0,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049247,1049247,19999828,29734428,NaN,30712109,2147-07-18 13:50:00,NaN,72.0,16.0,100.0,93.0,64.0,NaN,9,0
1049248,1049248,19999828,29734428,NaN,30712109,2147-07-18 16:35:00,99.6,78.0,17.0,99.0,108.0,87.0,NaN,NaN,0
1049249,1049249,19999987,23865745,NaN,34731548,2145-11-02 19:40:00,NaN,112.0,18.0,NaN,118.0,83.0,NaN,NaN,0
1049250,1049250,19999987,23865745,NaN,34731548,2145-11-02 20:11:00,NaN,111.0,18.0,NaN,123.0,82.0,NaN,unable,0


In [ ]:
data['charttime'] = pd.to_datetime(data['charttime'])
data['deathtime'] = pd.to_datetime(data['deathtime'])

In [ ]:
data['death'] = ((data['deathtime'] - data['charttime']).abs() <= pd.Timedelta(hours=1)).astype(int)

In [ ]:
data.head()

,hadm_id,temperature,heartrate,resprate,o2sat,sbp,dbp,death
0,22595853,97.7,79.0,16.0,98.0,107.0,60.0,0
1,22841357,97.9,76.0,18.0,95.0,95.0,64.0,0
2,22841357,97.9,86.0,17.0,93.0,96.0,57.0,0
3,25742920,98.5,96.0,17.0,100.0,102.0,58.0,0
4,25742920,98.1,91.0,18.0,99.0,98.0,60.0,0


In [ ]:
data = data.drop(columns=['subject_id', 'stay_id', 'Unnamed: 0', 'deathtime', 'charttime', 'rhythm', 'pain', 'hospital_expire_flag'])

In [ ]:
import numpy as np
numeric_columns = data.select_dtypes(include=[np.number]).columns

# 用列的平均值替换数值列中的NaN值
data[numeric_columns] = data[numeric_columns].apply(lambda x: x.fillna(x.mean()), axis=0)

In [ ]:
data.head()

,hadm_id,temperature,heartrate,resprate,o2sat,sbp,dbp,death
0,22595853,97.7,79.0,16.0,98.0,107.0,60.0,0
1,22841357,97.9,76.0,18.0,95.0,95.0,64.0,0
2,22841357,97.9,86.0,17.0,93.0,96.0,57.0,0
3,25742920,98.5,96.0,17.0,100.0,102.0,58.0,0
4,25742920,98.1,91.0,18.0,99.0,98.0,60.0,0


In [ ]:
mean_values = data.mean()

# 按hadm_id排序
data = data.sort_values(by='hadm_id').reset_index(drop=True)

# 填充缺失值
for col in data.columns:
    if col != 'hadm_id':  # 不处理hadm_id列
        data[col] = data.groupby('hadm_id')[col].ffill().fillna(mean_values[col])

In [ ]:
split_point = int(len(data) * 0.9)

# 创建前 90% 行的 DataFrame
df_90 = data.iloc[:split_point]

# 创建后 10% 行的 DataFrame
df_10 = data.iloc[split_point:]

# 输出两个 DataFrame
df_90, df_10

(         hadm_id  temperature  heartrate  resprate  o2sat    sbp   dbp  death
 0       22595853    97.700000       79.0      16.0   98.0  107.0  60.0      0
 1       22841357    97.900000       76.0      18.0   95.0   95.0  64.0      0
 2       22841357    97.900000       86.0      17.0   93.0   96.0  57.0      0
 3       25742920    98.500000       96.0      17.0  100.0  102.0  58.0      0
 4       25742920    98.100000       91.0      18.0   99.0   98.0  60.0      0
 ...          ...          ...        ...       ...    ...    ...   ...    ...
 944321  23624365    98.600000       78.0      18.0  100.0  148.0  78.0      0
 944322  23624365    97.800000       76.0      18.0  100.0  136.0  62.0      0
 944323  23624365    97.923266       64.0      16.0   99.0  113.0  75.0      0
 944324  23624365    98.000000       71.0      16.0  100.0  115.0  82.0      0
 944325  23624365    98.300000       70.0      15.0  100.0  125.0  59.0      0
 
 [944326 rows x 8 columns],
           hadm_id  te

In [ ]:
pip install imblearn

In [ ]:
from imblearn.over_sampling import SMOTE
X = df_90.drop('death', axis=1)
y = df_90['death']
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# 将重新采样后的特征和目标变量合并到一个新的dataframe中
resampled_data = pd.concat([X_resampled, y_resampled], axis=1)

In [ ]:
resampled_data

,hadm_id,temperature,heartrate,resprate,o2sat,sbp,dbp,death
0,22595853,97.700000,79.000000,16.000000,98.000000,107.000000,60.000000,0
1,22841357,97.900000,76.000000,18.000000,95.000000,95.000000,64.000000,0
2,22841357,97.900000,86.000000,17.000000,93.000000,96.000000,57.000000,0
3,25742920,98.500000,96.000000,17.000000,100.000000,102.000000,58.000000,0
4,25742920,98.100000,91.000000,18.000000,99.000000,98.000000,60.000000,0
...,...,...,...,...,...,...,...,...
1888633,24139497,87.862601,108.776990,31.107960,84.895585,73.122140,52.888495,1
1888634,25904580,97.923266,102.147533,22.834468,99.531449,103.867723,56.118950,1
1888635,26053074,97.950486,89.734728,18.065394,98.469888,123.547026,76.093036,1
1888636,25377349,97.923266,107.002366,24.007098,100.000000,98.004732,52.003549,1


In [ ]:
features = resampled_data.drop(columns=['hadm_id','death'])
labels = resampled_data['death']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix


X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# 4. 标准化数据
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 5. 模型训练
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train, y_train)

# 6. 模型评估
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"ROC AUC Score: {roc_auc}")
print(f"Confusion Matrix:\n{conf_matrix}")

Accuracy: 0.8807501694340901
ROC AUC Score: 0.9434391326588463
Confusion Matrix:
[[166201  22719]
 [ 22325 166483]]


In [ ]:
test_features = df_10.drop(columns=['hadm_id','death'])
test_labels = df_10['death']
t_test = scaler.transform(test_features)

y_pred = model.predict(t_test)
y_pred_proba = model.predict_proba(t_test)[:, 1]

accuracy = accuracy_score(test_labels, y_pred)
roc_auc = roc_auc_score(test_labels, y_pred_proba)
conf_matrix = confusion_matrix(test_labels, y_pred)

print(f"Accuracy: {accuracy}")
print(f"ROC AUC Score: {roc_auc}")
print(f"Confusion Matrix:\n{conf_matrix}")

Accuracy: 0.8778567752511294
ROC AUC Score: 0.8910504594151512
Confusion Matrix:
[[92102 12814]
 [    2     8]]


In [ ]:
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# 4. 标准化数据
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 5. 模型训练
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 6. 模型评估
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"ROC AUC Score: {roc_auc}")
print(f"Confusion Matrix:\n{conf_matrix}")

Accuracy: 0.9999920577770247
ROC AUC Score: 0.9999999997757194
Confusion Matrix:
[[188919      1]
 [     2 188806]]


In [ ]:
y_pred = model.predict(t_test)
y_pred_proba = model.predict_proba(t_test)[:, 1]

accuracy = accuracy_score(test_labels, y_pred)
roc_auc = roc_auc_score(test_labels, y_pred_proba)
conf_matrix = confusion_matrix(test_labels, y_pred)

print(f"Accuracy: {accuracy}")
print(f"ROC AUC Score: {roc_auc}")
print(f"Confusion Matrix:\n{conf_matrix}")

Accuracy: 0.9998856336846921
ROC AUC Score: 0.4886099355675016
Confusion Matrix:
[[104914      2]
 [    10      0]]


In [ ]:
pip install numpy pandas scikit-learn tensorflow

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader, Dataset

# Handle missing values by filling with the median of each column

# Select relevant features and target
features = [col for col in resampled_data.columns if col != 'hadm_id' and col != 'death']  # Exclude 'hamd_id' and target from features
scaler = MinMaxScaler()
resampled_data[features] = scaler.fit_transform(resampled_data[features])

# Create sequences for LSTM input based on hamd_id
def create_sequences_grouped_by_hamd_id(df, features, target):
    sequences = []
    for hamd_id, group in df.groupby('hadm_id'):
        group_X = group[features].values
        group_y = group[target].values[-1]  # Use the last value as the target
        sequences.append((group_X, group_y))
    return sequences

target_column = 'death'  # Replace with your actual target column name

sequences = create_sequences_grouped_by_hamd_id(resampled_data, features, target_column)

# Split sequences into features and targets
X_sequences = [seq[0] for seq in sequences]
y_sequences = [seq[1] for seq in sequences]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

# Custom Dataset class
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = [torch.tensor(seq, dtype=torch.float32) for seq in X]
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = TimeSeriesDataset(X_train, y_train)
test_dataset = TimeSeriesDataset(X_test, y_test)

def collate_fn(batch):
    batch_X, batch_y = zip(*batch)
    lengths = [len(x) for x in batch_X]
    padded_X = pad_sequence(batch_X, batch_first=True)
    batch_y = torch.tensor(batch_y, dtype=torch.float32)
    return padded_X, batch_y, lengths

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

# Define LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, lengths):
        h0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)
        c0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)
        packed_input = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        packed_output, (hn, cn) = self.lstm(packed_input, (h0, c0))
        out, _ = pad_packed_sequence(packed_output, batch_first=True)
        out = self.fc(out[:, -1, :])  # Take the output of the last time step
        out = self.sigmoid(out)
        return out

# Model parameters
input_size = len(features)
hidden_size = 50
num_layers = 2
output_size = 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMModel(input_size, hidden_size, num_layers, output_size).to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch, lengths in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        outputs = model(X_batch, lengths)
        loss = criterion(outputs, y_batch.unsqueeze(1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for X_batch, y_batch, lengths in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch, lengths)
        y_true.extend(y_batch.cpu().numpy())
        y_pred.extend(outputs.cpu().numpy())

# Calculate AUROC
y_pred = np.array(y_pred)
auroc = roc_auc_score(y_true, y_pred)
print(f'AUROC: {auroc:.4f}')

# Convert predictions to binary outcome for other evaluation metrics
y_pred_binary = y_pred > 0.5

# Print evaluation metrics
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_true, y_pred_binary))
print(classification_report(y_true, y_pred_binary))

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader, Dataset

# Handle missing values by filling with the median of each column

# Select relevant features and target
features = [col for col in resampled_data.columns if col != 'hadm_id' and col != 'death']  # Exclude 'hamd_id' and target from features
scaler = MinMaxScaler()
resampled_data[features] = scaler.fit_transform(resampled_data[features])

# Create sequences for LSTM input based on hamd_id
def create_sequences_grouped_by_hamd_id(df, features, target):
    sequences = []
    for hamd_id, group in df.groupby('hadm_id'):
        group_X = group[features].values
        group_y = group[target].values
        for i in range(1, len(group)):
            sequences.append((group_X[:i+1], group_y[i]))
    return sequences

target_column = 'death'  # Replace with your actual target column name

sequences = create_sequences_grouped_by_hamd_id(resampled_data, features, target_column)

# Split sequences into features and targets
X_sequences = [seq[0] for seq in sequences]
y_sequences = [seq[1] for seq in sequences]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

# Custom Dataset class
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = [torch.tensor(seq, dtype=torch.float32) for seq in X]
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = TimeSeriesDataset(X_train, y_train)
test_dataset = TimeSeriesDataset(X_test, y_test)

def collate_fn(batch):
    batch_X, batch_y = zip(*batch)
    lengths = [len(x) for x in batch_X]
    padded_X = pad_sequence(batch_X, batch_first=True)
    batch_y = torch.tensor(batch_y, dtype=torch.float32)
    return padded_X, batch_y, lengths

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

# Define LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, lengths):
        h0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)
        c0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)
        packed_input = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        packed_output, (hn, cn) = self.lstm(packed_input, (h0, c0))
        out, _ = pad_packed_sequence(packed_output, batch_first=True)
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        return out

# Model parameters
input_size = len(features)
hidden_size = 50
num_layers = 2
output_size = 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMModel(input_size, hidden_size, num_layers, output_size).to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch, lengths in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        outputs = model(X_batch, lengths)
        loss = criterion(outputs, y_batch.unsqueeze(1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for X_batch, y_batch, lengths in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch, lengths)
        y_true.extend(y_batch.cpu().numpy())
        y_pred.extend(outputs.cpu().numpy())

# Calculate AUROC
y_pred = np.array(y_pred)
auroc = roc_auc_score(y_true, y_pred)
print(f'AUROC: {auroc:.4f}')

# Convert predictions to binary outcome for other evaluation metrics
y_pred_binary = y_pred > 0.5

# Print evaluation metrics
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_true, y_pred_binary))
print(classification_report(y_true, y_pred_binary))

NameError: name 'resampled_data' is not defined

AUROC: 0.5000


array([0, 0, 0, ..., 0, 0, 0])